In [1]:
import pandas as pd
import numpy as np
import json
import requests
#from config import api_key
import yfinance as yf

#from pandas_datareader import data as pdr
#yf.pdr_override() 

In [7]:
# Read the raw data
path='Resources/key_stats_yahoo.csv'
df = pd.read_csv(path, index_col=False)

In [8]:
df

,Unnamed: 0,Date,Ticker,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
0,0,1/30/2004 19:01,a,33.8320,0.407,NaN,2.80,6.01,29.56,11.97,...,3.774,NaN,0.8,1.610000e+09,3.370,1.150000e+09,2.040,5.930,144000000.0,2.474
1,1,4/13/2004 4:07,a,28.4626,0.375,NaN,2.43,4.98,25.56,6.20,...,3.391,NaN,0.8,1.680000e+09,3.490,1.150000e+09,2.284,6.394,144000000.0,2.496
2,2,12/13/2004 21:37,a,21.8816,0.323,33.58,1.61,3.24,4.86,5.38,...,0.710,NaN,0.8,2.320000e+09,4.750,1.150000e+09,2.441,7.320,483000000.0,2.547
3,3,1/22/2005 5:17,a,20.2478,0.322,30.70,1.52,3.04,4.86,5.38,...,0.710,NaN,18.6,2.320000e+09,4.710,1.150000e+09,2.446,7.329,663000000.0,2.166
4,4,1/9/2006 2:35,a,31.7118,2.169,52.82,3.31,4.26,6.38,3.58,...,0.660,64.9,22.8,2.250000e+09,4.555,2.169000e+00,2.169,8.095,890000000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,9127,11/1/2012 16:37,znga,2.2300,5.390,NaN,1.37,0.94,46.51,47.90,...,0.890,NaN,3.2,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,NaN
9128,9128,11/10/2012 0:28,znga,2.2300,5.390,NaN,1.32,0.91,46.51,47.90,...,0.890,NaN,3.2,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,NaN
9129,9129,2/16/2013 1:09,znga,3.1700,5.480,NaN,1.96,1.37,16.35,6.83,...,0.280,NaN,NaN,1.280000e+09,1.670,1.000000e+08,2.920,2.370,195770000.0,3.940
9130,9130,4/22/2013 2:07,znga,3.2300,5.480,NaN,1.95,1.36,16.35,6.59,...,0.280,NaN,NaN,1.280000e+09,1.640,1.000000e+08,2.920,2.340,195770000.0,3.790


In [9]:
df = df.drop(columns=['Unnamed: 0'])

In [10]:
#Ticker Array
tickers = df['Ticker'].unique()
tickers

array(['a', 'aa', 'aapl', 'abbv', 'abc', 'abt', 'ace', 'aci', 'acn',
       'act', 'adbe', 'adi', 'adm', 'adp', 'adsk', 'adt', 'aee', 'aeo',
       'aep', 'aes', 'aet', 'afl', 'agn', 'aig', 'aiv', 'aiz', 'akam',
       'all', 'altr', 'alxn', 'amat', 'amd', 'amgn', 'amp', 'amt', 'amzn',
       'an', 'anf', 'ann', 'aon', 'apa', 'apc', 'apd', 'aph', 'apol',
       'arg', 'arna', 'aro', 'ati', 'atvi', 'avb', 'avp', 'avy', 'axp',
       'azo', 'ba', 'bac', 'bax', 'bbby', 'bbt', 'bby', 'bcr', 'bdx',
       'beam', 'ben', 'bhi', 'big', 'biib', 'bk', 'bks', 'blk', 'bll',
       'bms', 'bmy', 'brcm', 'bsx', 'btu', 'bwa', 'bxp', 'c', 'ca', 'cab',
       'cag', 'cah', 'cam', 'cat', 'cb', 'cbg', 'cbs', 'cce', 'cci',
       'ccl', 'celg', 'cern', 'cf', 'cfn', 'chk', 'chrw', 'ci', 'cim',
       'cinf', 'cl', 'clf', 'clx', 'cma', 'cmcsa', 'cme', 'cmg', 'cmi',
       'cms', 'cnp', 'cnx', 'cof', 'cog', 'coh', 'col', 'cop', 'cost',
       'cov', 'cpb', 'crm', 'csc', 'csco', 'csx', 'ctas', 'ctl', 'ctsh',

In [11]:
#List of number columns
num_col = df.dtypes[df.dtypes == 'float64'].index.tolist()
num_col

['Price',
 'DE Ratio',
 'Trailing P/E',
 'Price/Sales',
 'Price/Book',
 'Profit Margin',
 'Operating Margin',
 'Return on Assets',
 'Return on Equity',
 'Revenue Per Share',
 'Market Cap',
 'Enterprise Value',
 'Forward P/E',
 'PEG Ratio',
 'Enterprise Value/Revenue',
 'Enterprise Value/EBITDA',
 'Revenue',
 'Gross Profit',
 'EBITDA',
 'Net Income Avl to Common ',
 'Diluted EPS',
 'Earnings Growth',
 'Revenue Growth',
 'Total Cash',
 'Total Cash Per Share',
 'Total Debt',
 'Current Ratio',
 'Book Value Per Share',
 'Cash Flow',
 'Beta']

In [12]:
# data types
df.dtypes

Date                          object
Ticker                        object
Price                        float64
DE Ratio                     float64
Trailing P/E                 float64
Price/Sales                  float64
Price/Book                   float64
Profit Margin                float64
Operating Margin             float64
Return on Assets             float64
Return on Equity             float64
Revenue Per Share            float64
Market Cap                   float64
Enterprise Value             float64
Forward P/E                  float64
PEG Ratio                    float64
Enterprise Value/Revenue     float64
Enterprise Value/EBITDA      float64
Revenue                      float64
Gross Profit                 float64
EBITDA                       float64
Net Income Avl to Common     float64
Diluted EPS                  float64
Earnings Growth              float64
Revenue Growth               float64
Total Cash                   float64
Total Cash Per Share         float64
T

In [13]:
#Replace NaN with Mean
#ticker='a'
for ticker in tickers:
    m = df[df['Ticker'] == ticker][num_col].mean()
    mask = df[df['Ticker'] == ticker]
    mask[num_col] = mask[num_col].fillna(value = m)
    df[df['Ticker'] == ticker] = mask
df

C:\Users\levfl\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Date,Ticker,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
0,1/30/2004 19:01,a,33.8320,0.407,18.158125,2.80,6.01,29.56,11.97,24.95,...,3.774,114.269231,0.80,1.610000e+09,3.370,1.150000e+09,2.040,5.930,144000000.0,2.474000
1,4/13/2004 4:07,a,28.4626,0.375,18.158125,2.43,4.98,25.56,6.20,23.59,...,3.391,114.269231,0.80,1.680000e+09,3.490,1.150000e+09,2.284,6.394,144000000.0,2.496000
2,12/13/2004 21:37,a,21.8816,0.323,33.580000,1.61,3.24,4.86,5.38,5.26,...,0.710,114.269231,0.80,2.320000e+09,4.750,1.150000e+09,2.441,7.320,483000000.0,2.547000
3,1/22/2005 5:17,a,20.2478,0.322,30.700000,1.52,3.04,4.86,5.38,5.26,...,0.710,114.269231,18.60,2.320000e+09,4.710,1.150000e+09,2.446,7.329,663000000.0,2.166000
4,1/9/2006 2:35,a,31.7118,2.169,52.820000,3.31,4.26,6.38,3.58,1.68,...,0.660,64.900000,22.80,2.250000e+09,4.555,2.169000e+00,2.169,8.095,890000000.0,2.021353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,11/1/2012 16:37,znga,2.2300,5.390,128.105000,1.37,0.94,46.51,47.90,18.64,...,0.890,53.900000,3.20,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,3.840000
9128,11/10/2012 0:28,znga,2.2300,5.390,128.105000,1.32,0.91,46.51,47.90,18.64,...,0.890,53.900000,3.20,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,3.840000
9129,2/16/2013 1:09,znga,3.1700,5.480,128.105000,1.96,1.37,16.35,6.83,2.15,...,0.280,53.900000,49.36,1.280000e+09,1.670,1.000000e+08,2.920,2.370,195770000.0,3.940000
9130,4/22/2013 2:07,znga,3.2300,5.480,128.105000,1.95,1.36,16.35,6.59,2.07,...,0.280,53.900000,49.36,1.280000e+09,1.640,1.000000e+08,2.920,2.340,195770000.0,3.790000


In [15]:
#Check remaining Nans
df.isnull().sum()

Date                           0
Ticker                         0
Price                         59
DE Ratio                     490
Trailing P/E                  38
Price/Sales                    0
Price/Book                    27
Profit Margin                  0
Operating Margin               0
Return on Assets               0
Return on Equity               7
Revenue Per Share              0
Market Cap                     0
Enterprise Value               0
Forward P/E                   49
PEG Ratio                     18
Enterprise Value/Revenue       0
Enterprise Value/EBITDA      571
Revenue                        0
Gross Profit                   0
EBITDA                       571
Net Income Avl to Common       0
Diluted EPS                    0
Earnings Growth               44
Revenue Growth                 1
Total Cash                     0
Total Cash Per Share           0
Total Debt                     0
Current Ratio                462
Book Value Per Share           0
Cash Flow 

In [16]:
#df[df['DE Ratio'].isnull()]

In [17]:
#drop remaining Nans
df.dropna(axis = 'rows', how='any',inplace=True)
df

,Date,Ticker,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
0,1/30/2004 19:01,a,33.8320,0.407,18.158125,2.80,6.01,29.56,11.97,24.95,...,3.774,114.269231,0.80,1.610000e+09,3.370,1.150000e+09,2.040,5.930,144000000.0,2.474000
1,4/13/2004 4:07,a,28.4626,0.375,18.158125,2.43,4.98,25.56,6.20,23.59,...,3.391,114.269231,0.80,1.680000e+09,3.490,1.150000e+09,2.284,6.394,144000000.0,2.496000
2,12/13/2004 21:37,a,21.8816,0.323,33.580000,1.61,3.24,4.86,5.38,5.26,...,0.710,114.269231,0.80,2.320000e+09,4.750,1.150000e+09,2.441,7.320,483000000.0,2.547000
3,1/22/2005 5:17,a,20.2478,0.322,30.700000,1.52,3.04,4.86,5.38,5.26,...,0.710,114.269231,18.60,2.320000e+09,4.710,1.150000e+09,2.446,7.329,663000000.0,2.166000
4,1/9/2006 2:35,a,31.7118,2.169,52.820000,3.31,4.26,6.38,3.58,1.68,...,0.660,64.900000,22.80,2.250000e+09,4.555,2.169000e+00,2.169,8.095,890000000.0,2.021353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9127,11/1/2012 16:37,znga,2.2300,5.390,128.105000,1.37,0.94,46.51,47.90,18.64,...,0.890,53.900000,3.20,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,3.840000
9128,11/10/2012 0:28,znga,2.2300,5.390,128.105000,1.32,0.91,46.51,47.90,18.64,...,0.890,53.900000,3.20,1.320000e+09,1.690,1.000000e+08,2.680,2.380,339950000.0,3.840000
9129,2/16/2013 1:09,znga,3.1700,5.480,128.105000,1.96,1.37,16.35,6.83,2.15,...,0.280,53.900000,49.36,1.280000e+09,1.670,1.000000e+08,2.920,2.370,195770000.0,3.940000
9130,4/22/2013 2:07,znga,3.2300,5.480,128.105000,1.95,1.36,16.35,6.59,2.07,...,0.280,53.900000,49.36,1.280000e+09,1.640,1.000000e+08,2.920,2.340,195770000.0,3.790000


In [18]:
df.isnull().sum()

Date                         0
Ticker                       0
Price                        0
DE Ratio                     0
Trailing P/E                 0
Price/Sales                  0
Price/Book                   0
Profit Margin                0
Operating Margin             0
Return on Assets             0
Return on Equity             0
Revenue Per Share            0
Market Cap                   0
Enterprise Value             0
Forward P/E                  0
PEG Ratio                    0
Enterprise Value/Revenue     0
Enterprise Value/EBITDA      0
Revenue                      0
Gross Profit                 0
EBITDA                       0
Net Income Avl to Common     0
Diluted EPS                  0
Earnings Growth              0
Revenue Growth               0
Total Cash                   0
Total Cash Per Share         0
Total Debt                   0
Current Ratio                0
Book Value Per Share         0
Cash Flow                    0
Beta                         0
dtype: i

In [19]:
df.describe()

,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,Return on Equity,Revenue Per Share,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
count,8.345000e+03,8345.000000,8345.000000,8345.000000,8345.000000,8345.000000,8345.000000,8345.000000,8345.000000,8345.000000,...,8345.000000,8345.000000,8345.000000,8.345000e+03,8.345000e+03,8.345000e+03,8345.000000,8345.000000,8.345000e+03,8345.000000
mean,2.246744e+05,34.295701,29.190484,3.077368,5.245862,12.618482,17.701730,8.259699,27.574194,39.057522,...,3.308050,79.685804,26.759106,7.326090e+09,1.833691e+07,1.281523e+10,2.266766,18.057317,8.652418e+09,1.389150
std,2.060423e+06,82.441070,53.055592,14.890931,22.084525,13.086093,16.024305,6.220883,62.203166,46.625168,...,11.551713,232.877920,269.135844,5.421397e+10,4.585260e+08,4.929358e+10,5.582162,20.702655,2.905563e+10,3.515112
min,1.278000e-01,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.002000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.009000,0.000000,2.000000e+06,0.001000
25%,2.053220e+01,0.441000,13.560000,0.880000,1.810000,5.230000,8.510000,4.050000,11.250000,12.442000,...,1.220000,13.636364,5.400000,3.110000e+08,1.311000e+00,6.177000e+08,1.129000,7.821000,5.563000e+08,0.650000
50%,3.270970e+01,1.680000,18.750000,1.660000,2.930000,9.310000,14.460000,7.270000,17.740000,24.769000,...,2.204000,31.900000,11.100000,9.887700e+08,2.750000e+00,2.530000e+09,1.562000,13.662000,1.565561e+09,1.090000
75%,5.170040e+01,33.462500,26.840000,3.040000,4.740000,15.720000,22.640000,11.000000,26.230000,48.637000,...,3.750000,70.100000,21.200000,2.910000e+09,5.390000e+00,7.820000e+09,2.361000,22.310000,5.490000e+09,1.590000
max,7.639660e+07,987.070000,968.670000,732.160000,876.740000,223.470000,597.300000,113.670000,1642.390000,592.270000,...,508.130000,13667.900000,13219.300000,9.498000e+11,1.745000e+10,5.487000e+11,218.000000,472.609985,6.115200e+11,212.840000


In [20]:
#Remove outliers
#from scipy import stats
for i in num_col:
    df = df[np.abs(df[i]-df[i].mean()) <= (3*df[i].std())]

df = df[df['Price']<10000]
#cdf[(np.abs(stats.zscore(cdf)) < 3).all(axis=0)]
#cdf[(np.abs(stats.zscore(cdf[0])) < 3)]
df.describe()

,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,Return on Equity,Revenue Per Share,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
count,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,5461.000000,...,5461.000000,5461.000000,5461.000000,5.461000e+03,5.461000e+03,5.461000e+03,5461.000000,5461.000000,5.461000e+03,5461.000000
mean,37.584803,23.028505,24.650884,2.204068,3.837607,10.415157,15.640101,8.118199,20.049162,33.307688,...,2.410882,62.946148,14.571060,1.557439e+09,2.268518e+06,3.620854e+09,1.924717,15.393998,2.842347e+09,1.221581
std,38.980997,44.516702,19.523281,1.976997,3.321666,7.460080,9.591251,4.727978,15.289953,28.670031,...,1.688436,103.993435,13.847164,2.263683e+09,3.945928e+07,4.537063e+09,1.178667,10.039902,4.368913e+09,0.914680
min,1.408900,0.000000,1.710000,0.020000,0.080000,0.060000,0.040000,0.000000,0.000000,0.438000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.168000,0.000000,2.000000e+06,0.001000
25%,19.805000,0.393000,14.710000,0.900000,1.890000,5.040000,8.480000,4.430000,11.080000,13.666000,...,1.180000,13.300000,5.200000,2.557200e+08,1.220000e+00,4.915800e+08,1.150000,7.880000,4.917600e+08,0.679000
50%,30.740000,1.237556,19.620000,1.570000,2.980000,8.430000,13.550000,7.530000,17.180000,24.800000,...,1.990000,29.300000,10.500000,7.220300e+08,2.600000e+00,1.850000e+09,1.590000,13.159000,1.200000e+09,1.090000
75%,44.712800,27.130000,27.532143,2.790000,4.660000,14.000000,20.810000,10.890000,24.130000,44.323000,...,3.230000,63.900000,18.900000,1.990000e+09,4.620000e+00,5.010000e+09,2.307000,20.433001,3.440000e+09,1.570000
max,663.650800,281.390000,190.250000,22.090000,46.710000,49.330000,53.150000,26.130000,138.940000,181.520000,...,9.700000,811.600000,85.900000,1.616000e+10,1.140000e+09,2.523000e+10,7.318000,50.212002,3.163000e+10,12.230000


In [21]:
#export clean data
df.to_csv('Resources/clean_data.csv', index=False)

In [22]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder, MinMaxScaler
import pandas as pd
#import tensorflow as tf
#turn off the warnings
#from tensorflow 
import keras
import warnings
warnings.filterwarnings('ignore')

# Import checkpoint dependencies
import os
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import sklearn
import matplotlib.pyplot as plt

In [23]:
#import clean data  
# Make Index a Ticker

cdf0 = pd.read_csv('Resources/clean_data.csv', index_col=1)
cdf0

,Date,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,Return on Equity,...,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta
Ticker,,,,,,,,,,,,,,,,,,,,,
a,1/30/2004 19:01,33.8320,0.407,18.158125,2.80,6.01,29.56,11.97,24.95000,47.8100,...,3.774,114.269231,0.8,1.610000e+09,3.370,1.150000e+09,2.040,5.930,144000000.0,2.474000
a,4/13/2004 4:07,28.4626,0.375,18.158125,2.43,4.98,25.56,6.20,23.59000,46.8200,...,3.391,114.269231,0.8,1.680000e+09,3.490,1.150000e+09,2.284,6.394,144000000.0,2.496000
a,12/13/2004 21:37,21.8816,0.323,33.580000,1.61,3.24,4.86,5.38,5.26000,10.9200,...,0.710,114.269231,0.8,2.320000e+09,4.750,1.150000e+09,2.441,7.320,483000000.0,2.547000
a,1/22/2005 5:17,20.2478,0.322,30.700000,1.52,3.04,4.86,5.38,5.26000,10.9200,...,0.710,114.269231,18.6,2.320000e+09,4.710,1.150000e+09,2.446,7.329,663000000.0,2.166000
a,1/9/2006 2:35,31.7118,2.169,52.820000,3.31,4.26,6.38,3.58,1.68000,3.6900,...,0.660,64.900000,22.8,2.250000e+09,4.555,2.169000e+00,2.169,8.095,890000000.0,2.021353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
znga,12/30/2011 4:26,9.4100,5.422,126.620000,6.48,8.17,7.19,11.04,10.75125,29.5275,...,0.070,53.900000,79.8,9.256300e+08,1.320,0.000000e+00,1.590,1.160,283040000.0,3.840000
znga,1/11/2012 4:01,8.3300,5.422,115.270000,5.46,6.88,7.19,11.04,10.75125,29.5275,...,0.070,53.900000,79.8,9.256300e+08,1.320,0.000000e+00,1.590,1.160,283040000.0,3.840000
znga,3/11/2012 15:30,13.6700,5.422,128.105000,8.71,5.64,35.46,35.77,14.04000,36.2300,...,1.400,53.900000,59.0,1.810000e+09,2.490,0.000000e+00,3.030,2.420,389170000.0,3.840000


In [24]:
cdf = cdf0.drop(columns = ['Date',#'Ticker',#'DE Ratio',
    #'Trailing P/E',
    #'Price/Sales',
    #'Price/Book',
    #'Profit Margin',
    #'Operating Margin',
    #'Return on Assets',
    #'Return on Equity',
    #'Revenue Per Share',
    'Market Cap',
    'Enterprise Value',
    #'Forward P/E',
    #'PEG Ratio',
    #'Enterprise Value/Revenue',
    #'Enterprise Value/EBITDA',
    'Revenue',
    #'Gross Profit',
    'EBITDA',
    'Net Income Avl to Common ',
    #'Diluted EPS',
    #'Earnings Growth',
    #'Revenue Growth',
    'Total Cash',
    #'Total Cash Per Share',
    'Total Debt'#,
    #'Current Ratio',
    #'Book Value Per Share',
    #'Cash Flow',
    #'Beta'
                         ])
active_col = cdf.columns.to_list()
active_col

['Price',
 'DE Ratio',
 'Trailing P/E',
 'Price/Sales',
 'Price/Book',
 'Profit Margin',
 'Operating Margin',
 'Return on Assets',
 'Return on Equity',
 'Revenue Per Share',
 'Forward P/E',
 'PEG Ratio',
 'Enterprise Value/Revenue',
 'Enterprise Value/EBITDA',
 'Gross Profit',
 'Diluted EPS',
 'Earnings Growth',
 'Revenue Growth',
 'Total Cash Per Share',
 'Current Ratio',
 'Book Value Per Share',
 'Cash Flow',
 'Beta']

In [25]:
cdf.dtypes

Price                       float64
DE Ratio                    float64
Trailing P/E                float64
Price/Sales                 float64
Price/Book                  float64
Profit Margin               float64
Operating Margin            float64
Return on Assets            float64
Return on Equity            float64
Revenue Per Share           float64
Forward P/E                 float64
PEG Ratio                   float64
Enterprise Value/Revenue    float64
Enterprise Value/EBITDA     float64
Gross Profit                float64
Diluted EPS                 float64
Earnings Growth             float64
Revenue Growth              float64
Total Cash Per Share        float64
Current Ratio               float64
Book Value Per Share        float64
Cash Flow                   float64
Beta                        float64
dtype: object

In [26]:
# Split our preprocessed data into our features and target arrays
y = cdf["Price"].values
X = cdf.drop(["Price"],1).values

# Split the preprocessed data into a training and testing dataset
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)
X_scaled = X_scaler.transform(X)
#X_train_scaled

In [40]:
#Clustering of the Feature Dataset
import warnings
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas

warnings.filterwarnings('ignore')

# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X)
    inertia.append(km.inertia_)
    
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)



:Curve   [k]   (inertia)

In [29]:
n_c=6 # Number of Clusters

# Initialize the K-Means model.
model = KMeans(n_clusters=n_c, random_state=0)
model

# Fit the model
model.fit(X)

# Predict clusters
predictions = model.predict(X)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [30]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
cdf['Class'] = model.labels_

# Print the shape of the clustered_df
#print(clustered_df.shape)
cdf.to_csv('Resources/clean_data_clustered.csv')
cdf

,Price,DE Ratio,Trailing P/E,Price/Sales,Price/Book,Profit Margin,Operating Margin,Return on Assets,Return on Equity,Revenue Per Share,...,Gross Profit,Diluted EPS,Earnings Growth,Revenue Growth,Total Cash Per Share,Current Ratio,Book Value Per Share,Cash Flow,Beta,Class
Ticker,,,,,,,,,,,,,,,,,,,,,
a,33.8320,0.407,18.158125,2.80,6.01,29.56,11.97,24.95000,47.8100,12.803,...,2.290000e+09,3.774,114.269231,0.8,3.370,2.040,5.930,144000000.0,2.474000,0
a,28.4626,0.375,18.158125,2.43,4.98,25.56,6.20,23.59000,46.8200,13.160,...,2.290000e+09,3.391,114.269231,0.8,3.490,2.284,6.394,144000000.0,2.496000,0
a,21.8816,0.323,33.580000,1.61,3.24,4.86,5.38,5.26000,10.9200,14.610,...,2.290000e+09,0.710,114.269231,0.8,4.750,2.441,7.320,483000000.0,2.547000,0
a,20.2478,0.322,30.700000,1.52,3.04,4.86,5.38,5.26000,10.9200,14.610,...,3.120000e+09,0.710,114.269231,18.6,4.710,2.446,7.329,663000000.0,2.166000,0
a,31.7118,2.169,52.820000,3.31,4.26,6.38,3.58,1.68000,3.6900,10.403,...,3.120000e+09,0.660,64.900000,22.8,4.555,2.169,8.095,890000000.0,2.021353,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
znga,9.4100,5.422,126.620000,6.48,8.17,7.19,11.04,10.75125,29.5275,2.080,...,8.398200e+08,0.070,53.900000,79.8,1.320,1.590,1.160,283040000.0,3.840000,0
znga,8.3300,5.422,115.270000,5.46,6.88,7.19,11.04,10.75125,29.5275,2.080,...,8.398200e+08,0.070,53.900000,79.8,1.320,1.590,1.160,283040000.0,3.840000,0
znga,13.6700,5.422,128.105000,8.71,5.64,35.46,35.77,14.04000,36.2300,3.950,...,8.100600e+08,1.400,53.900000,59.0,2.490,3.030,2.420,389170000.0,3.840000,0


In [31]:
# Split datesets by Class

cdf =pd.read_csv('Resources/clean_data_clustered.csv', index_col=False)

cdf_clustered = []

for i in range(n_c):
    cdf_clustered.append(cdf[cdf['Class']==i])

cdf.dtypes

Ticker                       object
Price                       float64
DE Ratio                    float64
Trailing P/E                float64
Price/Sales                 float64
Price/Book                  float64
Profit Margin               float64
Operating Margin            float64
Return on Assets            float64
Return on Equity            float64
Revenue Per Share           float64
Forward P/E                 float64
PEG Ratio                   float64
Enterprise Value/Revenue    float64
Enterprise Value/EBITDA     float64
Gross Profit                float64
Diluted EPS                 float64
Earnings Growth             float64
Revenue Growth              float64
Total Cash Per Share        float64
Current Ratio               float64
Book Value Per Share        float64
Cash Flow                   float64
Beta                        float64
Class                         int64
dtype: object

In [32]:
for i in range(n_c):
    
    cdfi = cdf_clustered[i]
    cdfi = cdfi.drop(columns = ['Ticker','Class'])
    
    # Split our preprocessed data into our features and target arrays
    y = cdfi["Price"].values
    X = cdfi.drop(["Price"],1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)
    
    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    X_scaled = X_scaler.transform(X)  
    
    # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
    number_input_features = len(X_train_scaled[0])
    hidden_nodes_layer1 = 80
    hidden_nodes_layer2 = 30
    hidden_nodes_layer3 = 10

    #nn = tf.keras.models.Sequential()
    nn = keras.models.Sequential()

    # First hidden layer
    nn.add(keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu"))

    # Second hidden layer
    nn.add(keras.layers.Dense(units=hidden_nodes_layer2, activation="selu"))

    # Third hidden layer
    nn.add(keras.layers.Dense(units=hidden_nodes_layer3, activation="selu"))

    # Output layer
    nn.add(keras.layers.Dense(units=1, activation="relu"))

    # Check the structure of the model
    nn.summary()
    
    train_size = len(X_train_scaled)
    #train_size
    
    batch_size = 32
    steps_per_epoch = train_size / batch_size
    save_period = 5
    save_freq = int(save_period * steps_per_epoch)
    
    # Compile the model
    nn.compile(loss="mean_absolute_error", optimizer="adam")
    
    # Train the model
    fit_model = nn.fit(X_train_scaled, y_train, epochs=50, batch_size = batch_size)
    
    # Evaluate the model using the test data
    model_loss = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}")
    
    # Calculate Adjusted r-square
    import statsmodels.api as sm
    XaddC = sm.add_constant(X_test_scaled)
    result = sm.OLS(y_test, XaddC).fit()
    print(result.rsquared, result.rsquared_adj)
    
    # Export the model to HDF5 file
    nn.save(f"Model/Stock_Optimization_Class_{i}.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                1840      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 4,591
Trainable params: 4,591
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
82/82 [==============================] - 1s 2ms/step - loss: 28.6153
Epoch 2/50
82/82 [==============================] - 0s 2ms/step - loss: 22.6932
Epoch 3/50
82/82 [========

11/11 [==============================] - 0s 2ms/step - loss: 25.8565
Epoch 29/50
11/11 [==============================] - 0s 2ms/step - loss: 25.5662
Epoch 30/50
11/11 [==============================] - 0s 2ms/step - loss: 25.2936
Epoch 31/50
11/11 [==============================] - 0s 2ms/step - loss: 25.0126
Epoch 32/50
11/11 [==============================] - 0s 2ms/step - loss: 24.8774
Epoch 33/50
11/11 [==============================] - 0s 2ms/step - loss: 24.7917
Epoch 34/50
11/11 [==============================] - 0s 2ms/step - loss: 24.6293
Epoch 35/50
11/11 [==============================] - 0s 2ms/step - loss: 24.3799
Epoch 36/50
11/11 [==============================] - 0s 2ms/step - loss: 24.2087
Epoch 37/50
11/11 [==============================] - 0s 2ms/step - loss: 24.0422
Epoch 38/50
11/11 [==============================] - 0s 2ms/step - loss: 23.8484
Epoch 39/50
11/11 [==============================] - 0s 2ms/step - loss: 23.6994
Epoch 40/50
11/11 [=====================

3/3 [==============================] - 0s 2ms/step - loss: 41.5384
Epoch 5/50
3/3 [==============================] - 0s 3ms/step - loss: 41.1848
Epoch 6/50
3/3 [==============================] - 0s 3ms/step - loss: 40.8397
Epoch 7/50
3/3 [==============================] - 0s 2ms/step - loss: 40.4842
Epoch 8/50
3/3 [==============================] - 0s 4ms/step - loss: 40.1015
Epoch 9/50
3/3 [==============================] - 0s 2ms/step - loss: 39.7120
Epoch 10/50
3/3 [==============================] - 0s 3ms/step - loss: 39.3000
Epoch 11/50
3/3 [==============================] - 0s 3ms/step - loss: 38.8491
Epoch 12/50
3/3 [==============================] - 0s 3ms/step - loss: 38.3553
Epoch 13/50
3/3 [==============================] - 0s 3ms/step - loss: 37.8247
Epoch 14/50
3/3 [==============================] - 0s 3ms/step - loss: 37.2017
Epoch 15/50
3/3 [==============================] - 0s 2ms/step - loss: 36.5720
Epoch 16/50
3/3 [==============================] - 0s 3ms/step - loss

25/25 [==============================] - 0s 2ms/step - loss: 9.1671
Epoch 44/50
25/25 [==============================] - 0s 2ms/step - loss: 9.0336
Epoch 45/50
25/25 [==============================] - 0s 2ms/step - loss: 9.0805
Epoch 46/50
25/25 [==============================] - 0s 2ms/step - loss: 9.0447
Epoch 47/50
25/25 [==============================] - 0s 2ms/step - loss: 9.1652
Epoch 48/50
25/25 [==============================] - 0s 2ms/step - loss: 8.9380
Epoch 49/50
25/25 [==============================] - 0s 2ms/step - loss: 8.8948
Epoch 50/50
25/25 [==============================] - 0s 2ms/step - loss: 8.8063
9/9 - 0s - loss: 9.6695 - 108ms/epoch - 12ms/step
Loss: 9.669516563415527
0.42059383189622335 0.36611974771552636
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 80)                1840      
                                      